In [1]:
import time
import pandas as pd
import pymysql
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.firefox.options import Options
from selenium.webdriver.firefox.firefox_binary import FirefoxBinary
from selenium.webdriver.common.by import By
from sqlalchemy import create_engine
from sqlalchemy.ext.declarative import declarative_base
from sqlalchemy.orm import sessionmaker
import os


In [2]:
# Preparar conteúdo HTML a partir da URL
url = 'https://stats.nba.com/players/traditional/?PerMode=Totals&Season=2019-20&SeasonType=Regular%20Season&sort=PLAYER_NAME&dir=-1'
binary = FirefoxBinary('C:\\Program Files\\Mozilla Firefox\\firefox.exe')

option = Options()
option.headless = True
driver = webdriver.Firefox(firefox_binary=binary, executable_path='C:\\geckodriver.exe')

driver.get(url)
#time.sleep(10)

#element = driver.find_element_by_xpath("//div[@class='nba-stat-table']//table")
element = driver.find_element(By.XPATH, "//*[@class='Crom_table__p1iZz']")
html_content = element.get_attribute('outerHTML')
print(html_content)
time.sleep(5)

# Parsear conteúdo HTML - BeautifulSoup
soup = BeautifulSoup(html_content, 'lxml')
table = soup.find(name='table')
print(table)

C:\Users\andre\AppData\Local\Temp\ipykernel_29852\1677836944.py:6: DeprecationWarning: headless property is deprecated, instead use add_argument('-headless')
  option.headless = True
C:\Users\andre\AppData\Local\Temp\ipykernel_29852\1677836944.py:7: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Firefox(firefox_binary=binary, executable_path='C:\\geckodriver.exe')
C:\Users\andre\AppData\Local\Temp\ipykernel_29852\1677836944.py:7: DeprecationWarning: firefox_binary has been deprecated, please pass in a Service object
  driver = webdriver.Firefox(firefox_binary=binary, executable_path='C:\\geckodriver.exe')


<table class="Crom_table__p1iZz"><thead><tr class="Crom_headers__mzI_m"><th class="Crom_stickyRank__aN66p">&nbsp;</th><th field="PLAYER_NAME" class="Crom_text__NpR1_ Crom_primary__EajZu Crom_stickySecondColumn__29Dwf">Player</th><th sort="true" field="TEAM_ABBREVIATION" class="Crom_text__NpR1_">Team</th><th sort="true" field="AGE">Age</th><th sort="true" field="GP" rank="true" title="Games Played">GP</th><th sort="true" field="W" rank="true" title="Wins">W</th><th sort="true" field="L" rank="true" title="Losses">L</th><th sort="true" field="MIN" rank="true" title="Minutes Played">Min</th><th sort="true" field="PTS" rank="true" sorted="A" title="Points">PTS</th><th sort="true" field="FGM" rank="true" title="Field Goals Made">FGM</th><th sort="true" field="FGA" rank="true" title="Field Goals Attempted">FGA</th><th sort="true" field="FG_PCT" rank="true" title="Field Goal Percentage">FG%</th><th sort="true" field="FG3M" rank="true" title="3 Point Field Goals Made">3PM</th><th sort="true" f

In [10]:
# Estrurar conteúdo em um Data Frame - Pandas
df_full = pd.read_html(str(table))
df_sql = pd.DataFrame(df_full[0])
df_sql.drop(columns=['Unnamed: 0'], inplace=True)


print(df_sql)

print(type(df_sql))

df_sql.columns

                     Player Team  Age  GP   W   L   Min   PTS   FGM   FGA   
0               Joel Embiid  PHI   29  66  43  23  34.6  33.1  11.0  20.1  \
1               Luka Doncic  DAL   24  66  33  33  36.2  32.4  10.9  22.0   
2            Damian Lillard  POR   32  58  27  31  36.3  32.2   9.6  20.7   
3   Shai Gilgeous-Alexander  OKC   24  68  33  35  35.5  31.4  10.4  20.3   
4     Giannis Antetokounmpo  MIL   28  63  47  16  32.1  31.1  11.2  20.3   
5              Jayson Tatum  BOS   25  74  52  22  36.9  30.1   9.8  21.1   
6             Stephen Curry  GSW   35  56  30  26  34.7  29.4  10.0  20.2   
7              Kevin Durant  PHX   34  47  34  13  35.6  29.1  10.3  18.3   
8              LeBron James  LAL   38  55  30  25  35.5  28.9  11.1  22.2   
9          Donovan Mitchell  CLE   26  68  44  24  35.8  28.3  10.0  20.6   
10             Devin Booker  PHX   26  53  34  19  34.6  27.8   9.9  20.1   
11             Kyrie Irving  DAL   31  60  32  28  37.3  27.1   9.9  20.1   

Index(['Player', 'Team', 'Age', 'GP', 'W', 'L', 'Min', 'PTS', 'FGM', 'FGA',
       'FG%', '3PM', '3PA', '3P%', 'FTM', 'FTA', 'FT%', 'OREB', 'DREB', 'REB',
       'AST', 'TOV', 'STL', 'BLK', 'PF', 'FP', 'DD2', 'TD3', '+/-', 'GP RANK',
       'W RANK', 'L RANK', 'MIN RANK', 'PTS RANK', 'FGM RANK', 'FGA RANK',
       'FG% RANK', '3PM RANK', '3PA RANK', '3P% RANK', 'FTM RANK', 'FTA RANK',
       'FT% RANK', 'OREB RANK', 'DREB RANK', 'REB RANK', 'AST RANK',
       'TOV RANK', 'STL RANK', 'BLK RANK', 'PF RANK', 'FP RANK', 'DD2 RANK',
       'TD3 RANK', '+/- RANK'],
      dtype='object')

In [5]:
df_full[0]

,Unnamed: 0,Player,Team,Age,GP,W,L,Min,PTS,FGM,...,REB RANK,AST RANK,TOV RANK,STL RANK,BLK RANK,PF RANK,FP RANK,DD2 RANK,TD3 RANK,+/- RANK
0,1,Joel Embiid,PHI,29,66,43,23,34.6,33.1,11.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2,Luka Doncic,DAL,24,66,33,33,36.2,32.4,10.9,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,3,Damian Lillard,POR,32,58,27,31,36.3,32.2,9.6,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,4,Shai Gilgeous-Alexander,OKC,24,68,33,35,35.5,31.4,10.4,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,5,Giannis Antetokounmpo,MIL,28,63,47,16,32.1,31.1,11.2,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,6,Jayson Tatum,BOS,25,74,52,22,36.9,30.1,9.8,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,7,Stephen Curry,GSW,35,56,30,26,34.7,29.4,10.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,8,Kevin Durant,PHX,34,47,34,13,35.6,29.1,10.3,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,9,LeBron James,LAL,38,55,30,25,35.5,28.9,11.1,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,10,Donovan Mitchell,CLE,26,68,44,24,35.8,28.3,10.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
